In [31]:
import time as clock
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange
from IPython.display import clear_output, display
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

In [32]:
# configuration of simulator
# and parametrs of reservouir
perm = np.load('perm.npy')
nx0, nx1 = perm.shape
nx2 = 1
perm = np.reshape(perm, (nx0, nx1, nx2))
poro = 0.1 + np.zeros((nx0, nx1, nx2))

dx0 = 1.0 / nx0
dx1 = 1.0 / nx1
dx2 = 1.0 / nx2

pwat = 2.0
poil = 4.0
vr = 0.3
kwat = 1.0
koil = 0.3

pmin = 0.0
pmax = 1.0

niter = 5000
t_final = 3.0
dt = t_final / niter

In [33]:
nx0, nx1

(64, 64)

# Read from file

In [34]:
# sim_200 = np.random.randint(0, 5000, 200)
# sim_500 = np.random.randint(0, 5000, 500)
# sim_1000 = np.random.randint(0, 5000, 1000)
# sim_2000 = np.random.randint(0, 5000, 2000)
# np.save('sim_200.npy', sim_200)
# np.save('sim_500.npy', sim_500)
# np.save('sim_1000.npy', sim_1000)
# np.save('sim_2000.npy', sim_2000)

In [84]:
N = 100
sim_r = np.load(f'sim_{N}.npy')

# 5000
sim_data = np.loadtxt("data_5k_200225/sim_5000.txt", dtype=np.float32)[sim_r]
x_list = np.loadtxt("data_5k_200225/x_5000.txt", dtype=np.float32)[sim_r]
y_list = np.loadtxt("data_5k_200225/y_5000.txt", dtype=np.float32)[sim_r]
t_list = np.loadtxt("data_5k_200225/t_5000.txt", dtype=np.float32)[sim_r]
# sim_data, x_list, y_list, t_list

In [85]:
sim_data[:, 1].mean()

np.float32(0.042194746)

In [86]:
simulation_data = torch.tensor(sim_data).requires_grad_(True)

x = torch.tensor(x_list * dx0).requires_grad_(True)
y = torch.tensor(y_list * dx1).requires_grad_(True)
t = torch.tensor(t_list * dt).requires_grad_(True)

points = torch.stack((t, x, y), -1).requires_grad_(True)
# points, simulation_data

# sim
# pres
# swat
# soil

In [87]:
DEVICE = 'cpu'

In [88]:
class PINN(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=32, num_layers=8, output_dim=7):
        super().__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.U = nn.Linear(input_dim, hidden_dim)
        self.V = nn.Linear(input_dim, hidden_dim)

        self.hidden_layers = nn.ModuleList(
            [nn.Linear(hidden_dim, hidden_dim) for _ in range(num_layers - 1)]
        )

        self.output_layer = nn.Linear(hidden_dim, output_dim)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight, gain=1.0)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        U = torch.tanh(self.U(x))
        V = torch.tanh(self.V(x))

        h = U * V

        for i, layer in enumerate(self.hidden_layers):
            h_new = torch.relu(layer(h))
            if i % 2 == 1 and i > 0:
                h = h + h_new
            else:
                h = h_new

        T = self.output_layer(h)
        return T

# SVM

In [89]:
def check_current_situation_others(svm_model, scaler, t):
    x = np.linspace(0, 1, 100)
    time = t * np.ones(100)
    real_temp = thermal_conductivity_equation([torch.tensor(time), torch.tensor(x)])

    test_points = np.stack((time, x), axis = -1)

    svm_pred = svm_model.predict(scaler.transform(test_points))

    plt.plot(x, svm_pred, label=f'SVM prediction with t = {t}')
    plt.plot(x, real_temp, label=f'Analytical solution with t = {t}')
    plt.grid()
    plt.xlabel('x')
    plt.ylabel(f'T(t={t}, x)')

    plt.plot()
    plt.legend()
    plt.show()
    
def check_current_situation_xgb(xgb, t):
    x = np.linspace(0, 1, 100)
    time = t * np.ones(100)
    real_temp = thermal_conductivity_equation([torch.tensor(time), torch.tensor(x)])

    test_points = np.stack((time, x), axis = -1)

    svm_pred = xgb.predict(test_points)

    plt.plot(x, svm_pred, label=f'SVM prediction with t = {t}')
    plt.plot(x, real_temp, label=f'Analytical solution with t = {t}')
    plt.grid()
    plt.xlabel('x')
    plt.ylabel(f'T(t={t}, x)')

    plt.plot()
    plt.legend()
    plt.show()

In [90]:
X_train = points.cpu().detach().numpy()
y_train = simulation_data.cpu().detach().numpy()

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

svr = SVR(kernel="rbf", C=1.0, epsilon=0.1, gamma="scale")
model_svr = MultiOutputRegressor(svr)
model_svr.fit(X_train, y_train)
# y_pred = svr.predict(X_test_scaled)
# print("MSE:", mean_squared_error(y_test, y_pred))
# print("R^2:", r2_score(y_test, y_pred))

MultiOutputRegressor(estimator=SVR())

In [91]:
# check_current_situation_others(svr, scaler, t=0.0)
# check_current_situation_others(svr, scaler, t=0.01)
# check_current_situation_others(svr, scaler, t=0.02)
# check_current_situation_others(svr, scaler, t=0.03)
# check_current_situation_others(svr, scaler, t=0.04)
# check_current_situation_others(svr, scaler, t=0.05)

# XGBoost

In [92]:
# Example data
xgb_regressor = xgb.XGBRegressor(tree_method='hist', verbosity=2)
xgb_regressor.fit(X_train, y_train)

[02:03:28] INFO: /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (100, 3, 300).


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [93]:
# check_current_situation_xgb(xgb_regressor, t=0)
# check_current_situation_xgb(xgb_regressor, t=0.01)
# check_current_situation_xgb(xgb_regressor, t=0.02)
# check_current_situation_xgb(xgb_regressor, t=0.03)
# check_current_situation_xgb(xgb_regressor, t=0.04)
# check_current_situation_xgb(xgb_regressor, t=0.05)

# Gaussian Processes Regression

In [94]:
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)) \
         + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True)
gp.fit(X_train, y_train)
# y_pred, sigma = gp.predict(X_test, return_std=True)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=10, normalize_y=True)

In [95]:
# check_current_situation_xgb(gp, t=0)
# check_current_situation_xgb(gp, t=0.01)
# check_current_situation_xgb(gp, t=0.02)
# check_current_situation_xgb(gp, t=0.03)
# check_current_situation_xgb(gp, t=0.04)
# check_current_situation_xgb(gp, t=0.05)

# Metrics Computation

In [96]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error, 
    r2_score, 
    median_absolute_error, 
    mean_absolute_percentage_error
)

def evaluate_multidim_models(models_dict, X_test, y_test, device='cpu'):
    """
    Evaluates models on multi-output regression (Target dim: [N, 3]).
    Output Columns: Pressure, Soil (Oil), Swat (Water).
    
    Parameters:
    - models_dict: Dictionary {'ModelName': model_object}
    - X_test: Test features
    - y_test: Test targets (Must be shape [N, 3])
    - device: 'cpu' or 'cuda' (for PyTorch model)
    
    Returns:
    - pd.DataFrame: Rows are (Variable_Metric), Columns are model names.
    """
    results = {}
    
    # 1. Define the specific output names
    target_names = ["Pressure", "Soil", "Swat"]
    
    # 2. Prepare Ground Truth (y_true) as numpy [N, 3]
    if torch.is_tensor(y_test):
        y_true = y_test.detach().cpu().numpy()
    elif isinstance(y_test, (pd.DataFrame, pd.Series)):
        y_true = y_test.to_numpy()
    else:
        y_true = np.array(y_test)
        
    # Basic shape validation
    if y_true.ndim != 2 or y_true.shape[1] != 3:
        raise ValueError(f"y_test must be shape [N, 3], got {y_true.shape}")

    # 3. Evaluate each model
    for name, model in models_dict.items():
        y_pred = None
        
        # --- Generate Predictions [N, 3] ---
        
        # PyTorch Logic
        if isinstance(model, nn.Module):
            model.eval()
            
            # Prepare input tensor
            if torch.is_tensor(X_test):
                X_tensor = X_test.to(device)
            elif isinstance(X_test, (pd.DataFrame, pd.Series)):
                X_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
            else:
                X_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
            
            with torch.no_grad():
                raw_pred = model(X_tensor)[:, [0, 2, 1]]
                y_pred = raw_pred.cpu().numpy() # Keep shape [N, 3]
                
        # Scikit-Learn Logic
        else:
            if torch.is_tensor(X_test):
                X_input = X_test.cpu().numpy()
            else:
                X_input = X_test
                
            y_pred = model.predict(X_input) # Expecting [N, 3] output

        # Ensure prediction shape matches truth
        if y_pred.shape != y_true.shape:
             raise ValueError(f"Model {name} output shape {y_pred.shape} mismatch with y_test {y_true.shape}")

        # --- Calculate Metrics per Column ---
        model_metrics = {}
        
        # Loop over the 3 output dimensions
        for i, label in enumerate(target_names):
            # Extract single column (1D arrays)
            y_t = y_true[:, i]
            y_p = y_pred[:, i]
            
            # Calculate Standard Metrics
            mse = mean_squared_error(y_t, y_p)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_t, y_p)
            r2 = r2_score(y_t, y_p)
            medae = median_absolute_error(y_t, y_p)
            mape = mean_absolute_percentage_error(y_t, y_p)
            
            # Store with prefixed keys (e.g., "Pressure_MSE")
            model_metrics[f"{label}_MSE"] = mse
            model_metrics[f"{label}_RMSE"] = rmse
            model_metrics[f"{label}_MAE"] = mae
            model_metrics[f"{label}_R2"] = r2
            model_metrics[f"{label}_MedAE"] = medae
            model_metrics[f"{label}_MAPE"] = mape

        results[name] = model_metrics

    # Create DataFrame
    df_results = pd.DataFrame(results)
    
    return df_results

In [97]:
pinn = PINN()
checkpoint = torch.load(f"pinn_model_weights_{20}.pth", map_location='cpu')
pinn.load_state_dict(checkpoint)
print(f"pinn_model_weights_{N}.pth")

pinn_model_weights_100.pth


In [68]:
N = 500
sim_r = np.load(f'sim_{N}.npy')

# 5000
sim_data = np.loadtxt("data_5k_200225/sim_5000.txt", dtype=np.float32)[sim_r]
x_list = np.loadtxt("data_5k_200225/x_5000.txt", dtype=np.float32)[sim_r]
y_list = np.loadtxt("data_5k_200225/y_5000.txt", dtype=np.float32)[sim_r]
t_list = np.loadtxt("data_5k_200225/t_5000.txt", dtype=np.float32)[sim_r]
# sim_data, x_list, y_list, t_list

In [69]:
sim_data[:, 1].mean()

np.float32(0.028082374)

In [70]:
simulation_data = torch.tensor(sim_data).requires_grad_(True)

x = torch.tensor(x_list * dx0).requires_grad_(True)
y = torch.tensor(y_list * dx1).requires_grad_(True)
t = torch.tensor(t_list * dt).requires_grad_(True)

points = torch.stack((t, x, y), -1).requires_grad_(True)
# points, simulation_data

# sim
# pres
# swat
# soil

In [71]:
X_test = points.detach().numpy()
y_test = simulation_data.detach().numpy()[:, [0, 1, 2]]

In [72]:
my_models = {
    'PINN': pinn,
    'XGB': xgb_regressor,
    'SVM': model_svr,
    'GP': gp
}

df_metrics = evaluate_multidim_models(my_models, X_test, y_test)
df_metrics

,PINN,XGB,SVM,GP
Pressure_MSE,1.633230e-02,6.129065e-04,4.517062e-03,2.752149e-03
Pressure_RMSE,1.277979e-01,2.475695e-02,6.720909e-02,5.246093e-02
Pressure_MAE,7.374001e-02,1.616435e-02,5.813688e-02,3.236428e-02
Pressure_R2,8.528722e-01,9.944787e-01,9.593085e-01,9.752076e-01
Pressure_MedAE,3.862599e-02,9.764522e-03,5.802537e-02,1.748386e-02
Pressure_MAPE,3.382962e-01,6.650846e-02,3.851198e-01,5.640009e-01
Soil_MSE,9.119167e-03,1.630281e-03,5.937355e-03,2.645198e-03
Soil_RMSE,9.549433e-02,4.037674e-02,7.705423e-02,5.143149e-02
Soil_MAE,3.818797e-02,1.441434e-02,6.323049e-02,2.406183e-02
Soil_R2,-2.113308e-01,7.834440e-01,2.113203e-01,6.486290e-01
